In [1]:
!python --version

Python 3.8.0


In [2]:
# !poetry remove pandas

In [3]:

# !poetry add tensorflow=2.5 tensorflow-text=2.5 tensorflow-hub 
# numpy=1.19.3 
# pandas==1.2.5 openpyxl

!poetry add pandas==1.2.5 openpyxl

^C


In [1]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

from scipy import spatial

In [2]:
# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [3]:
# Compute embeddings.
en_result = embed(english_sentences)
it_result = embed(italian_sentences)
ja_result = embed(japanese_sentences)

In [4]:
en_result.numpy().shape

(3, 512)

In [5]:
# Compute similarity matrix. Higher score indicates greater similarity.
similarity_matrix_it = np.inner(en_result, it_result)
similarity_matrix_ja = np.inner(en_result, ja_result)

In [6]:
similarity_matrix_it

array([[0.957878  , 0.3308628 , 0.30247933],
       [0.38761458, 0.73392963, 0.2481403 ],
       [0.23617546, 0.21800505, 0.92830104]], dtype=float32)

In [7]:
import pandas as pd

In [8]:
df = pd.read_excel("ЭТ_Новости.xlsx")
df.head()

,ID,NEWS_HEAD
0,1552621575,Государственная Дума рекомендует Правительству...
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...


In [9]:
%%time
df["emb"] = df["NEWS_HEAD"].apply(lambda x: embed(x).numpy()[0])

Wall time: 9.89 s


In [10]:
df.head()

,ID,NEWS_HEAD,emb
0,1552621575,Государственная Дума рекомендует Правительству...,"[6.6673114e-05, 0.02740392, -0.033854015, -0.0..."
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...,"[0.014879701, -0.0015657819, 0.031823713, -0.0..."
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...,"[-0.024322681, 0.063832596, 0.006030567, -0.04..."
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...,"[0.029627595, 0.06477559, -0.015955612, 0.0087..."
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...,"[-0.028674196, -0.037707165, -0.0035214513, -0..."


In [12]:
df.to_pickle("../News+emb.pkl")

In [13]:
emb_ar = np.stack(df.emb.to_numpy())
emb_ar.shape

(1975, 512)

In [15]:
def get_closest_to_text(text, df):
    print(f"Input string:\n{text}\n")
    emb_ar = np.stack(df.emb.to_numpy())
    text_emb = embed(text)
    cos_dist = np.array([1-spatial.distance.cosine(text_emb, x) for x in emb_ar])
    print("Closest record:")
    return df.iloc[np.argmax(cos_dist)]

In [16]:
get_closest_to_text(df.iloc[0].NEWS_HEAD, df)

Input string:
Государственная Дума рекомендует Правительству РФ, несмотря на кризис, вызванный санкциями недружественных стран, сохранить рыночное ценообразование на нефтепродукты и электроэнергию на внутреннем рынке и продолжить разработку мер поддержки новых отраслей ТЭК, связанных с переходом на низкоуглеродную траекторию развития экономики.

Closest record:


ID                                                  1552621575
NEWS_HEAD    Государственная Дума рекомендует Правительству...
emb          [6.6673114e-05, 0.02740392, -0.033854015, -0.0...
Name: 0, dtype: object

In [17]:
get_closest_to_text(df.iloc[2].NEWS_HEAD, df)

Input string:
Павел Сниккарс: «Важная задача в электроэнергетике в период высокой турбулентности – сохранить отлаженные за 10 лет механизмы»

Closest record:


ID                                                  1552115549
NEWS_HEAD    Павел Сниккарс: «Важная задача в электроэнерге...
emb          [-0.024322681, 0.063832596, 0.006030567, -0.04...
Name: 2, dtype: object

In [24]:
get_closest_to_text(df.iloc[18].NEWS_HEAD, df)

Input string:
ФАС определила задачи совершенствования тарифного регулирования

Closest record:


ID                                                  1546783300
NEWS_HEAD    ФАС определила задачи совершенствования тарифн...
emb          [0.09777064, 0.026426591, 0.051717404, -0.0283...
Name: 18, dtype: object

In [23]:
get_closest_to_text(df.iloc[19].NEWS_HEAD, df)

Input string:
ФАС определила задачи совершенствования тарифного регулирования

Closest record:


ID                                                  1546783300
NEWS_HEAD    ФАС определила задачи совершенствования тарифн...
emb          [0.09777064, 0.026426591, 0.051717404, -0.0283...
Name: 18, dtype: object

In [18]:
get_closest_to_text(df.iloc[142].NEWS_HEAD, df)

Input string:
Уведомление для ООО "Грин Энерджи Рус"! Информация о максимально допустимых величинах потребления на собственные нужды электростанций на 2021 год!

Closest record:


ID                                                  1497423434
NEWS_HEAD    Уведомление для ООО "Грин Энерджи Рус"! Информ...
emb          [-0.006868089, -0.011465344, 0.038518388, -0.0...
Name: 142, dtype: object

In [19]:
get_closest_to_text("I love pizza", df)

Input string:
I love pizza

Closest record:


ID                                                  1506307579
NEWS_HEAD                                   С Днем энергетика!
emb          [0.03395263, 0.027581275, -0.021881483, 0.0482...
Name: 124, dtype: object